In [47]:
import pandas as pd
import numpy as np
from sklearn import ensemble, preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import StratifiedKFold
from sklearn import ensemble, preprocessing, grid_search, cross_validation




In [48]:
ls

124.ipynb               imp.txt                 test.csv
Blend_op/               imp_feat.txt            test5fold_lol.csv
Latest best/            sample.csv              train.csv
Mergeop.ipynb           sample1.csv             train1.csv
Untitled.ipynb          sample123.csv           train5fold_lol.csv
Untitled1.ipynb         sample2.csv             xgb.fmap
Untitled2.ipynb         sample3.csv             xgbmodel.txt
blend.csv               sample_submissions.csv


In [49]:
train = pd.read_csv('train.csv')#,nrows = 100)
test = pd.read_csv('test.csv')#,nrows =100)

In [50]:
[i for i in train.columns if i not in test.columns]

['target']

In [51]:
train.shape

(348978, 51)

In [52]:
train.head(10)

,transaction_id,num_var_1,num_var_2,num_var_3,num_var_4,num_var_5,num_var_6,num_var_7,cat_var_1,cat_var_2,...,cat_var_34,cat_var_35,cat_var_36,cat_var_37,cat_var_38,cat_var_39,cat_var_40,cat_var_41,cat_var_42,target
0,id_11,2.302632e-08,0.040182,0.0,1.800000e-07,2.302632e-08,2.368421e-08,1.115205e-08,NaN,ce,...,0,0,0,0,0,0,0,0,0,0
1,id_33,7.965789e-06,0.157872,0.0,2.105000e-06,2.769737e-07,7.965789e-06,2.433058e-06,da,tn,...,0,0,0,0,0,0,0,0,0,0
2,id_51,7.828947e-08,0.089140,0.0,3.550000e-07,4.671053e-08,1.052632e-07,4.276014e-07,gf,ce,...,0,0,0,0,0,0,0,0,0,0
3,id_54,7.894737e-08,0.227239,0.0,1.050000e-06,1.381579e-07,2.190789e-07,1.848054e-08,NaN,ce,...,0,0,0,0,0,0,0,0,0,0
4,id_62,3.321053e-06,0.160410,0.0,2.105000e-06,2.769737e-07,3.340789e-06,2.152983e-06,da,tn,...,0,0,0,0,0,0,0,0,0,0
5,id_67,1.953947e-07,0.078566,0.0,3.550000e-07,4.671053e-08,4.407895e-08,9.463310e-08,gf,ce,...,0,0,0,0,0,0,0,0,0,0
6,id_71,1.736842e-07,0.600508,0.0,4.575000e-06,5.986842e-07,1.736842e-07,8.411831e-08,hn,ce,...,0,0,0,0,0,0,0,0,0,0
7,id_88,5.789474e-08,0.152374,0.0,2.105000e-06,2.769737e-07,5.789474e-08,2.453451e-08,da,tn,...,0,0,0,0,0,0,0,0,0,0
8,id_95,3.289474e-08,0.093053,0.0,7.500000e-07,8.223684e-08,9.868421e-08,1.274520e-08,pu,ce,...,0,0,0,0,0,0,0,0,0,0
9,id_97,5.500000e-07,0.158401,0.0,2.105000e-06,2.769737e-07,5.500000e-07,3.138505e-07,da,tn,...,0,0,0,0,0,0,0,0,0,0


In [53]:
train.target.value_counts()[0]*1.0/train.target.value_counts()[1]

8.3389531149646761

In [54]:
#for i in train.columns:
#   if train[i].dtype != 'object':
#        continue
#    print len(np.unique(np.union1d(np.unique(test[i]),np.unique(train[i])))), len(np.setdiff1d(np.unique(test[i]),np.unique(train[i])))

In [55]:
col_more = []
col_less = []
col_num = []
train.replace(np.nan,'ksj',inplace= True)
test.replace(np.nan,'ksj',inplace= True)
for i in train.columns:
    if train[i].dtype == 'object':
        if len(np.unique(list(train[i])+list(test[i]))) >30:
            col_more.append(i)
        else:
            col_less.append(i)
        #print i, len(np.unique(list(train[i])+list(test[i]))),len(np.setdiff1d(np.unique(test[i]),np.unique(train[i])))
    else:
        col_num.append(i)

In [56]:
col_more.remove('transaction_id')

In [57]:
def consolidate(df, col, threshold = 20):
    raw = df[col].copy()
    cnt = raw.value_counts()
    low_cardinal = list(cnt[cnt < threshold].index)
    raw[raw.isin(low_cardinal)] = "OTHER"
    return np.array(raw)

In [58]:
"""
c = ['num_var_1','num_var_2','num_var_3','num_var_4','num_var_5','num_var_6','num_var_7']
for i in c:    
    print i
    df = pd.concat((train[[i]],test[[i]]))
    vals = df[i].copy().dropna().sort_values().round(4)
    vals = pd.rolling_apply(vals, 2, lambda x: x[1] - x[0])
    vals = vals[vals > 1e-3]
    denom = vals.value_counts().idxmax() 
    train[i] = np.round(np.array(train[i]/denom),0).astype(int)
    test[i] = np.round(np.array(test[i]/denom),0).astype(int)
"""

"\nc = ['num_var_1','num_var_2','num_var_3','num_var_4','num_var_5','num_var_6','num_var_7']\nfor i in c:    \n    print i\n    df = pd.concat((train[[i]],test[[i]]))\n    vals = df[i].copy().dropna().sort_values().round(4)\n    vals = pd.rolling_apply(vals, 2, lambda x: x[1] - x[0])\n    vals = vals[vals > 1e-3]\n    denom = vals.value_counts().idxmax() \n    train[i] = np.round(np.array(train[i]/denom),0).astype(int)\n    test[i] = np.round(np.array(test[i]/denom),0).astype(int)\n"

In [59]:
for enu,i in enumerate(col_more):
    train[i] = consolidate(train,i)
    test[i] = consolidate(test,i)
    for j in col_more[enu+1:]:
        print i,j
        total = pd.concat((train[['transaction_id',i,j]],test[['transaction_id',i,j]]))
        k = total.groupby([i,j]).agg('count').reset_index()
        k.columns = [i,j,i+'_'+j+'_count'] 
        k1 = k.groupby([i]).agg('count')[j].reset_index()
        k1.columns = [i,i+'_'+j+'_count1']
        k2 = k.groupby([j]).agg('count')[i].reset_index()
        k2.columns = [j,j+'_'+i+'_count1']
        train = train.merge(k,on=[i,j],how='left')
        test = test.merge(k,on=[i,j],how='left')
        train = train.merge(k1,on=[i],how='left')
        test = test.merge(k1,on=[i],how='left')
        train = train.merge(k2,on=[j],how='left')
        test = test.merge(k2,on=[j],how='left')
    
    
    
        
#train = train.drop(col_more,axis=1)
#test = test.drop(col_more,axis=1)




cat_var_1 cat_var_2
cat_var_1 cat_var_3
cat_var_1 cat_var_6
cat_var_1 cat_var_8
cat_var_1 cat_var_13
cat_var_2 cat_var_3
cat_var_2 cat_var_6
cat_var_2 cat_var_8
cat_var_2 cat_var_13
cat_var_3 cat_var_6
cat_var_3 cat_var_8
cat_var_3 cat_var_13
cat_var_6 cat_var_8
cat_var_6 cat_var_13
cat_var_8 cat_var_13


In [60]:
"""
col_num.remove('target')
from itertools import combinations
for p,q in list(combinations(range(len(col_num)),2)):
    print p,q
    i = col_num[p]
    j = col_num[q]
    train[str(i)+str(j)] = train[i]*train[j]
    test[str(i)+str(j)] = test[i]*test[j]
        
"""


"\ncol_num.remove('target')\nfrom itertools import combinations\nfor p,q in list(combinations(range(len(col_num)),2)):\n    print p,q\n    i = col_num[p]\n    j = col_num[q]\n    train[str(i)+str(j)] = train[i]*train[j]\n    test[str(i)+str(j)] = test[i]*test[j]\n        \n"

In [61]:
y = train.target.values
id_test = test.transaction_id.values


In [62]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
text_columns = []
for f in train.columns:
    if train[f].dtype== 'object': 
        text_columns.append(f)            
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train[f].values) + list(test[f].values))
        train[f] = lbl.transform(list(train[f].values))
        test[f] = lbl.transform(list(test[f].values)) 
train = train.drop(['transaction_id','target'],axis =1)
test = test.drop(['transaction_id'],axis =1)

In [63]:
import operator
def ceate_feature_map(features):
    outfile = open('xgb.fmap', 'w')
    i = 0
    for feat in features:
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
        i = i + 1

    outfile.close()


In [64]:
import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.metrics import  roc_auc_score,roc_auc_score
def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=0,max_depth=10):
        params = {}
        params["objective"] = "binary:logistic"
        params['eval_metric'] = 'auc'
        params["eta"] = 0.01 #0.00334
        params["min_child_weight"] = 1
        params["subsample"] = 0.8
        params["colsample_bytree"] = 0.3
        params["silent"] = 1
        params["max_depth"] = max_depth
        params["seed"] = seed_val
        #params["max_delta_step"] = 2
        params["verbose"] = 100
        num_rounds = 5000 #2500

        plst = list(params.items())
        xgtrain = xgb.DMatrix(train_X, label=train_y)

        if test_y is not None:
                xgtest = xgb.DMatrix(test_X, label=test_y)
                watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
                model = xgb.train(plst , xgtrain, num_rounds, watchlist, early_stopping_rounds= 500)
        else:
                xgtest = xgb.DMatrix(test_X)
                #xgtest1 = xgb.DMatrix(test_X1)
                model = xgb.train(plst, xgtrain, 2000)
        #print feature_names,if feature_names
        if feature_names:
                        create_feature_map(feature_names)
                        model.dump_model('xgbmodel.txt', 'xgb.fmap', with_stats=True)
                        importance = model.get_fscore(fmap='xgb.fmap')
                        importance = sorted(importance.items(), key=operator.itemgetter(1), reverse=True)
                        imp_df = pd.DataFrame(importance, columns=['feature','fscore'])
                        imp_df['fscore'] = imp_df['fscore'] / imp_df['fscore'].sum()
                        imp_df.to_csv("imp_feat.txt", index=False)

        pred_test_y = model.predict(xgtest)
        
        if test_y is not None:
                loss = roc_auc_score(test_y, pred_test_y)
                print loss
        	return pred_test_y, loss
	else:
		return pred_test_y

In [ ]:
"""
#0.729290022223
#0.729594073648
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.3, random_state=42)

#xgb_test = np.array(test_final)
#0.603178481377
k = runXGB(X_train, y_train, X_test,y_test)
"""


'\n#0.729290022223\n#0.729594073648\nfrom sklearn.model_selection import train_test_split\n\nX_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.3, random_state=42)\n\n#xgb_test = np.array(test_final)\n#0.603178481377\nk = runXGB(X_train, y_train, X_test,y_test)\n'

In [ ]:
pred1 = runXGB(train, y, test,max_depth=10)
pd.DataFrame({'transaction_id':id_test,'target':pred1})[['transaction_id','target']].to_csv('sample1.csv',index =False)

print ('yes')
pred2 = runXGB(train, y, test,max_depth=12)
pd.DataFrame({'transaction_id':id_test,'target':pred2})[['transaction_id','target']].to_csv('sample2.csv',index =False)

pred3 = runXGB(train, y, test,max_depth=6)
pd.DataFrame({'transaction_id':id_test,'target':pred2})[['transaction_id','target']].to_csv('sample3.csv',index =False)


yes


In [ ]:
pred = (pred1+pred2+pred3)*1.0/3
pd.DataFrame({'transaction_id':id_test,'target':pred})[['transaction_id','target']].to_csv('sample.csv',index =False)

In [ ]:
pred

In [ ]:
denormalized

In [ ]:
train.head(10)